In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import os
import matplotlib.image as img
import tqdm
from vae import Basic_VAE
from encoder import Encoder
from decoder import Decoder

#### Create Dataset


In [7]:
cats = []
directory = "data/cats"
count = 0
for catpic in os.listdir(directory):
    if count < 1000:
        # read from image and convert to tensor
        im = torch.tensor(img.imread(os.path.join(directory, catpic))).float()
        # permute to (channels, height, width) for conv2d layer
        im = torch.permute(im, (2, 0, 1))
        # normalize to range between -1 and 1
        im = im / 128 - 1
        cats.append(im)
        count += 1
    else:
        break
cats = torch.stack(cats)
print(cats.shape)

torch.Size([10, 3, 64, 64])


#### Test/Training/Validation Split


In [8]:
# Split the data into what we use for testing and not testing
training_test_split = 0.75
training_test_cutoff = int(cats.shape[0] * training_test_split + 1)
random_perm = torch.randperm(cats.shape[0])
not_test_tensor = cats[:training_test_cutoff]
testing_tensor = cats[training_test_cutoff:]
# Split the data into what we use for training and cross validation
training_cv_split = 0.8
training_cv_cutoff = int(not_test_tensor.shape[0] * training_cv_split)
training_tensor = not_test_tensor[:training_cv_cutoff]
cv_tensor = not_test_tensor[training_cv_cutoff:]
print(training_tensor.shape)
print(cv_tensor.shape)
print(testing_tensor.shape)

torch.Size([6, 3, 64, 64])
torch.Size([2, 3, 64, 64])
torch.Size([2, 3, 64, 64])


#### Choose Hyperparameters and Build Model


In [9]:
from vae import Basic_VAE
from encoder import Encoder
from decoder import Decoder

hidden_dims = [16, 32, 64, 128]
latent_dim = 64
in_dim = 3
model = Basic_VAE(in_dim, hidden_dims, latent_dim)
# encoder = Encoder(in_dim, hidden_dims, latent_dim) for testing
# decoder = Decoder(latent_dim, hidden_dims) for testing

[128, 64, 32, 16]
[128, 64, 32, 16]


In [12]:
# # Testing the model
model.forward(training_tensor)

# # Code to test out the encoder & decoder
# mu, log_var = encoder.forward(training_tensor)
# print("mu: ", mu.shape)
# print("log_var: ", log_var.shape)

# reconstructed_img = decoder.forward(mu, log_var)

encoded.shape: torch.Size([6, 128, 3, 3])
encoded.shape after flatten: torch.Size([6, 1152])


[tensor([[[[ 6.2127e-01,  4.3679e-01, -4.9945e-01,  ...,  8.9123e-01,
             6.2095e-01, -1.6943e-01],
           [-9.7679e-01, -2.5637e-01,  2.3110e-01,  ...,  1.8704e-01,
            -2.1503e-01, -4.8144e-01],
           [-5.5388e-01, -7.7562e-01, -5.7911e-01,  ...,  3.5442e-01,
            -5.2527e-02, -2.8206e-01],
           ...,
           [ 6.9180e-01, -1.6230e-02, -1.3026e-01,  ...,  7.1330e-01,
            -1.6798e-01,  5.3718e-02],
           [-2.5497e-01, -4.2844e-01, -8.0627e-01,  ...,  2.2175e-01,
            -4.0637e-01, -5.3341e-02],
           [-1.1091e-01,  1.4218e-02, -1.7719e-01,  ..., -3.7198e-01,
             6.8662e-01, -1.2087e-01]],
 
          [[-7.3256e-01, -9.6734e-01, -8.7966e-01,  ...,  5.1233e-01,
            -1.3779e-02,  1.2253e-01],
           [-8.6216e-02, -8.1402e-01,  8.7467e-01,  ..., -7.8543e-01,
             8.2743e-01,  1.4651e-01],
           [-3.3732e-01, -6.4741e-01, -5.7643e-01,  ..., -2.0220e-01,
             1.5155e-01, -1.9310e-01],


#### Implement Loss Function


In [23]:
def loss_function(reconstructed_img, input_img, mu, log_var, kld_weight=2):
    print("reconstructed_img: ", reconstructed_img.shape)
    print("input_img: ", input_img.shape)
    img_loss = F.mse_loss(reconstructed_img, input_img)
    # article on calculating kl divergence between 2 gaussians:
    # https://medium.com/@outerrencedl/variational-autoencoder-and-a-bit-kl-divergence-with-pytorch-ce04fd55d0d7
    kld_loss = torch.mean(
        torch.sum(-log_var + (log_var.exp() ** 2 + mu**2) / 2 - 1 / 2)
    )
    kld_loss *= kld_weight

    return img_loss + kld_loss

#### Implement Gradient Descent


In [19]:
def gradient_descent(model, loss_func, x, y, xvalid, yvalid, lr=0.1, steps=5000):
    # only really need x or y, they are the same thing
    optimizer = optim.AdamW(model.parameters(), lr)

    losses = []
    valid_losses = []
    for _ in tqdm.trange(steps):
        model.train()
        reconstructed_img, mu, log_var = model(x)
        # y is the original image I think?
        loss = loss_func(reconstructed_img, y, mu, log_var)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        model.eval()
        reconstructed_img, mu, log_var = model(xvalid)
        valid_loss = loss_func(reconstructed_img, yvalid, mu, log_var)
        losses.append(loss.detach().numpy())
        valid_losses.append(valid_loss.detach().numpy())

    print(f"Final training loss: {losses[-1]}")

    return losses, valid_losses

In [26]:
losses, valid_losses = gradient_descent(
    model, loss_function, training_tensor, training_tensor, cv_tensor, cv_tensor
)

  0%|          | 0/5000 [00:00<?, ?it/s]

encoded.shape: torch.Size([6, 128, 3, 3])
encoded.shape after flatten: torch.Size([6, 1152])


/var/folders/j3/ll6yjvb96yjb1gn7mk4x45zh0000gn/T/ipykernel_59731/241648366.py:4: UserWarning: Using a target size (torch.Size([6, 3, 64, 64])) that is different to the input size (torch.Size([6, 3, 8, 8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  img_loss = F.mse_loss(reconstructed_img, input_img)
  0%|          | 0/5000 [06:13<?, ?it/s]

reconstructed_img:  torch.Size([6, 3, 8, 8])
input_img:  torch.Size([6, 3, 64, 64])


RuntimeError: The size of tensor a (8) must match the size of tensor b (64) at non-singleton dimension 3

In [ ]:
# after we've trained, we use the decoder to generate new images
# we assume a normal distribution over our latent space
# so we sample from that distribution and feed it into the decoder